# Creating a modified version of the AWRA-L model

In this example, we will create a local copy of AWRAL, and modify this copy to replace the existing (modelled) albedo with input data (eg satellite derived)


In [ ]:
# Import the libraries required for this notebook

from awrams.utils import config_manager
import os
from os.path import join
import shutil
from awrams.simulation import ondemand
import pandas as pd
from awrams.utils import datetools
from awrams.utils import extents
from awrams.utils.nodegraph import nodes

In [ ]:
# Since we're basing this on version 6, with modifications relating to albedo, let's call it 'v6_albmod'

model_name = 'v6_albmod'

## Creating copies

There are 2 components that we need to modify; the configuration files, and the core model code (C code)<br>
Open the existing model profile and have a look at where it stores its data

In [ ]:
sys_profile = config_manager.get_system_profile()
sys_settings = sys_profile.get_settings()

In [ ]:
model_profile = config_manager.get_model_profile('awral','v6_default')
model_settings = model_profile.get_settings()

In [ ]:
# Have a look at our existing system settings paths...

sys_settings.DATA_PATHS

In [ ]:
# Config files live just below AWRAMS_BASE

model_config_base = join(sys_settings.DATA_PATHS.AWRAMS_BASE,'config/models/awral')
os.listdir(model_config_base)

In [ ]:
# Typically one might use a terminal or file manager to make the copies
# cp /config_file_path/v6_default.py /config_file_path/v6_albmod.py
# For convenience you can just run this cell instead

shutil.copyfile(join(model_config_base,'v6_default.py'),join(model_config_base,'v6_albmod.py'))

In [ ]:
# The BUILD_SETTINGS item in model_settings contains information about where this model reads its C code from
# CORE_SRC_PATH is the one we're interested in

model_settings.BUILD_SETTINGS

In [ ]:
# Again, it may be more natural to use a terminal for this
# cp -r /source_code_path/v6 /source_code_path/v6_albmod

base_src_path, src_version = os.path.split(model_settings.BUILD_SETTINGS.CORE_SRC_PATH)
shutil.copytree(model_settings.BUILD_SETTINGS.CORE_SRC_PATH,join(base_src_path,model_name))

## Required modifications

There are 2 files to modify for this<br>

1. ***Configuration file*** [v6_albmod.py]
2. ***C code*** [awral_t.c]

[v6_albmod.py]: ../../awrams/config/models/awral/v6_albmod.py
[awral_t.c]: ../../awrams/code/models/awral/v6_albmod/awral_t.c

### Edit configuration [model_config_path] (editable)
[model_config_path]: ../../awrams/config/models/awral/
[v6_albmod.py]: ../../awrams/config/models/awral/v6_albmod.py

There are 3 changes required in the configuration file

1. ***Update the MODEL_VERSION string to point to our new code***
2. ***Add albedo input to the MODEL_INPUTS dictionary***
3. ***Change the input mapping to supply a default value for our new input*** 

### 1. Change the version string

Modify the MODEL_VERSION string (line 32); we want to change this to reflect our new model name<br>
This string is used to find the C source code files
   
#### Existing [v6_albmod.py]

    32: 'MODEL_VERSION' : 'v6'
    
#### New [v6_albmod.py]

    32: 'MODEL_VERSION' : 'v6_albmod'
    
[v6_albmod.py]: ../../awrams/config/models/awral/v6_albmod.py

### 2. Add new input in MODEL_INPUTS

There are various types of inputs specified in this dictionary:

  - ***STATES_CELL***  - model states that have a single value in a grid/catchment cell (do not differ by HRU)
  - ***STATES_HRU***   - model states that have different values on a HRU basis value in a grid/catchment cell 
  - ***INPUTS_CELL***  - model inputs that have a single value in a grid/catchment cell (do not differ by HRU)
  - ***INPUTS_HRU***   - model inputs that have different values on an HRU basis (that may also vary by cell)
  - ***INPUTS_HYPSO*** - inputs related to the Hypsometric curves
  
The type of data (ie scalar, spatial, or forcing (timeseries)) is computed from the input map, so does not need to be specified here

Insert the following line (at line 69) to the INPUTS_CELL section

    "alb": "Surface albedo",



#### Existing [v6_albmod.py]

    68: "INPUTS_CELL": {
    69:     "avpt": "Vapour pressure",
    70:     "k0sat": "Hydraulic saturation (top)",
    71:     "k_gw": "Groundwater drainage coefficient",
        ...
        
#### New [v6_albmod.py]

    68: "INPUTS_CELL": {
    69:     "alb": "Surface albedo",
    70:     "avpt": "Vapour pressure",
    71:     "k0sat": "Hydraulic saturation (top)",
        ...
        
### 3. Update the input mapping to supply a value for our new input variable

We need to supply a value for alb in the get_input_mapping method of the config file (starts line 183)<br>
We'll add the following line just under mapping['pair'] since they're both constants<br>
mapping['alb'] = nodes.const(0.3)

#### Existing [v6_albmod.py]

    261: mapping['pair'] = nodes.const(97500.)
    262:
    
#### New [v6_albmod.py]
    
    261: mapping['pair'] = nodes.const(97500.)
    262: mapping['alb'] = nodes.const(0.3)
    263:
    

# 2. Modify core code [source_path] (editable)

### Remove existing code block

Comment out the section of the main model code ([awral_t.c]) that contains the current albedo calculation<br>
The dynamic compiler will automatically generate the code for receiving the new input data; this is all you need to do!

    229: double alb_veg = 0.452 * vc;
    230: double alb_soil = alb_wet + (alb_dry - alb_wet) * exp(-w0 / w0ref_alb);
    231: double alb = fveg * alb_veg + fsoil * alb_soil;
    232: double rsn = (1.0 - alb) * rgeff;

Comment out old albedo calculation code by using // ...

    229: //double alb_veg = 0.452 * vc;
    230: //double alb_soil = alb_wet + (alb_dry - alb_wet) * exp(-w0 / w0ref_alb);
    231: //double alb = fveg * alb_veg + fsoil * alb_soil;
    232: double rsn = (1.0 - alb) * rgeff;
    
[awral_t.c]: ../../awrams/code/models/awral/v6_albmod/awral_t.c
[source_path]: ../../awrams/code/models/awral/v6_albmod/

### Examine the new model

In [ ]:
model_profile = config_manager.get_model_profile('awral',model_name)
model_settings = model_profile.get_settings()

In [ ]:
# Observe how the new paths have been set up
model_settings.BUILD_SETTINGS

### Running the model

Here we run 2 versions of the model; v6_default, and our new v6_albmod<br>

In [ ]:
# Specify the models we're interested in; by using their profile names we can easily iterate over multiple
# models

models = ['v6_default','v6_albmod']

In [ ]:
period = datetools.dates('2009-2011')
extent = extents.get_default_extent().ioffset[200:210,200:210]

res = {}

for m in models:
    model_profile = config_manager.get_model_profile('awral', m)
    imap = model_profile.get_input_mapping()
    model = model_profile.get_model()
    ods = ondemand.OnDemandSimulator(model,imap)
    res[m] = ods.run(period,extent)

In [ ]:
# Set up matplotlib

%matplotlib inline

import matplotlib
matplotlib.rcParams['figure.figsize'] = [16,6]

In [ ]:
#Albedo most directly affects potential evapotranspiration, so we'll examine e0

df = pd.DataFrame(index=period)

for m in models:
    df[m] = res[m]['e0'][:,0,0]

df.plot()

In [ ]:
(df['v6_albmod'] - df['v6_default']).plot()

In [ ]:
# Now we'll just run our fixed albedo model, but with some more extreme values of the alb constant

In [ ]:
alb_comp_res = {}

alb_values = [0.1,0.5,0.9]

for alb in alb_values:
    model_profile = config_manager.get_model_profile('awral', 'v6_albmod')
    imap = model_profile.get_input_mapping()
    
    # Observe 
    imap.alb = nodes.const(alb)
    
    model = model_profile.get_model()
    ods = ondemand.OnDemandSimulator(model,imap)
    alb_comp_res[alb] = ods.run(period,extent)

In [ ]:
#See how this effects actual evapotranspiration over time

df = pd.DataFrame(index=period)

for alb in alb_values:
    df[alb] = alb_comp_res[alb]['etot'][:,0,0]

df.plot()

### Further excercises

<br>
Change the input mapping for 'alb' to point to use real data instead of a const<br><br>
Remove redundant code - which model inputs are now unnecessary?